# Tarea 2
## Alumnos:
1. Ángel Bachler Pacheco
2. Manuel José Reyes Molinari


# Setup

In [12]:
!pip install pyspark
!pip install neo4j

In [13]:
import copy
import itertools
import pandas as pd
from neo4j import GraphDatabase

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [15]:
URI = "neo4j+s://802d1a9c.databases.neo4j.io"
AUTH = ("neo4j","INSERTAR KEY")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    try:
        session.run("RETURN 1")
        print("Connection to Neo4j established successfully!")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

Connection to Neo4j established successfully!


In [16]:
def nodos_relaciones():
    with driver.session() as session:
        #Consulta
        query = "MATCH (x)-[R]->(y) RETURN x, R, y"
        results = session.run(query)
        nodos = []
        relaciones = []
        #Guardamos los nodos y sus relaciones
        for record in results:
            nodos.append(record['x'])
            nodos.append(record['y'])
            relaciones.append(record['R'])
        # Eliminamos los nodos duplicados
        nodos = list({node.element_id: node for node in nodos}.values())
        lista = [[rel.start_node.id, rel.type, rel.end_node.id] for rel in relaciones]
        return lista

In [17]:
rdd = sc.parallelize(nodos_relaciones())
rdd.collect()

<ipython-input-16-1065cd0b1d81>:15: DeprecationWarning: `id` is deprecated, use `element_id` instead
  lista = [[rel.start_node.id, rel.type, rel.end_node.id] for rel in relaciones]


[[0, 'S', 1],
 [0, 'S', 2],
 [1, 'S', 2],
 [2, 'S', 3],
 [3, 'S', 0],
 [3, 'S', 1],
 [4, 'R', 0],
 [4, 'R', 1],
 [4, 'R', 5]]

In [18]:
driver.close()

## Parte 1

In [19]:
# Loading example data
example_data = [(1,11,2),(1,11,3),(2,11,3),(3,11,2),(3,11,4),(4,11,1),(4,11,2),(4,11,3),(4,12,5),(5,12,1),(5,12,2),(5,12,6)]
#example_data = [(4,11,1),(4,11,2),(4,12,5),(5,12,1),(5,12,2)]
#example_data = [(2,11,3),(3,11,4),(4,11,3),(3,11,2)]
#example_data = [(1,11,2),(4,11,1),(2,11,3),(3,11,4),(4,11,3),(3,11,2)]
rdd = sc.parallelize(example_data)
rdd.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6)]

In [20]:
def h_hash(vertex:int, b:int = 3) -> int:
  return vertex%b

In [21]:
class Edge:
  def __init__(self, vertex1, edge, vertex2) -> None:
    self.vertex1 = vertex1
    self.edge = edge
    self.vertex2 = vertex2

  def __hash__(self) -> int:
    return (self.vertex1, self.edge, self.vertex2).__hash__()

  def __eq__(self, other) -> bool:
    if not isinstance(other, Edge):
      return NotImplemented
    return self.vertex1 == other.vertex1 and self.edge == other.edge and self.vertex2 == other.vertex2

  def __repr__(self) -> str:
    return f"[{self.vertex1}, {self.edge}, {self.vertex2}]"

In [22]:
def pattern_reader(A : list, L : list, M : list[list[list]]) -> list[Edge]:
    pattern = []
    for x in range(len(M)):
      for R in range(len(M[x])):
        for y in range(len(M[x][R])):
          if M[x][R][y] == 1:
            pattern.append(Edge(A[x], L[R], A[y]))

    return pattern

In [23]:
A = ["a", "b", "c"]
L = [11, 12]
M = [
    [
        [0,1,0],
        [0,0,0]
    ],
    [
        [0,0,1],
        [0,0,0]
    ],
    [
        [1,0,0],
        [0,0,0]
    ]
]

pattern_reader(A, L, M)

[[a, 11, b], [b, 11, c], [c, 11, a]]

In [24]:
def calculate_key(edge:Edge, pattern:list[Edge], b:int=3) -> list[tuple[tuple[int,int,int],Edge]]:
  results = []
  b1 = h_hash(edge.vertex1, b)
  b2 = h_hash(edge.vertex2, b)
  for pattern_edge in pattern:
    if edge.edge == pattern_edge.edge:
      # How many "holes" to fill in the key
      n = len(pattern) - 2
      combinations = list(itertools.product(range(b),repeat=n))
      combinations = [list(c) for c in combinations]
      for key in combinations:
        # Insert low vertex first to avoid errors
        if pattern_edge.vertex1 < pattern_edge.vertex2:
          key.insert(pattern_edge.vertex1, b1)
          key.insert(pattern_edge.vertex2, b2)
        else:
          key.insert(pattern_edge.vertex2, b2)
          key.insert(pattern_edge.vertex1, b1)
        results.append((tuple(key),edge))

  return list(set(results))

def process_pattern(pattern:list[Edge]):
  vertices = []
  for edge in pattern:
    if edge.vertex1 not in vertices:
      vertices.append(edge.vertex1)
      edge.vertex1 = len(vertices) - 1
    else:
      edge.vertex1 = vertices.index(edge.vertex1)
    if edge.vertex2 not in vertices:
      vertices.append(edge.vertex2)
      edge.vertex2 = len(vertices) - 1
    else:
      edge.vertex2 = vertices.index(edge.vertex2)
  return len(vertices)


In [25]:
pattern = [Edge('a', 11, 'b'),Edge('b', 11, 'c'),Edge('c', 11, 'a')]
#pattern = [Edge("x",11,"y"), Edge("y",11,"z"), Edge("z",11,"w"), Edge("w",11,"x")]
#pattern = [Edge("x",11,"y"), Edge("y",11,"z"), Edge("z",12,"w"), Edge("w",12,"x")]
#pattern_vertex_count = process_pattern(pattern)

In [26]:
class EdgeCombination:
  def __init__(self, pattern_vertex_count:int, pattern_edge_count:int):
    self.pattern_vertex_count = pattern_vertex_count
    self.pattern_edge_count = pattern_edge_count
    self.edges:list[Edge] = []
    self.vertices:list = []

  def __repr__(self) -> str:
    return f"Edges: {self.edges}\nVertices: {self.vertices}\n----------------\n"

  def check_edge(self, new_edge:Edge) -> bool:
    # Avoid storing more edges than the pattern allows
    if len(self.edges) < self.pattern_edge_count:
      # Avoid storing more vertices than the pattern allows
      needed_space = 2
      if new_edge.vertex1 in self.vertices:
        needed_space -= 1
      if new_edge.vertex2 in self.vertices:
        needed_space -= 1
      if self.pattern_vertex_count >= len(self.vertices) + needed_space:
        return True

    return False

  def add_edge(self, new_edge:Edge):
    self.edges.append(new_edge)
    if new_edge.vertex1 not in self.vertices:
      self.vertices.append(new_edge.vertex1)
    if new_edge.vertex2 not in self.vertices:
      self.vertices.append(new_edge.vertex2)

  def check_if_pattern_is_matched(self, pattern:list[Edge]) -> bool:
    # Check if at least one of the ordering of the edges matches the pattern
    n = len(self.edges)
    if n == len(pattern):
      edge_orderings = itertools.permutations(self.edges)
      for ordering in edge_orderings:
        valid_ordering = True
        vertex_assigments = dict()
        for i, edge in enumerate(ordering):
          # Check that edge matches
          if pattern[i].edge == edge.edge:
            # Validate that both vertices are valid
            valid_v1 = pattern[i].vertex1 not in vertex_assigments or vertex_assigments[pattern[i].vertex1] == edge.vertex1
            valid_v2 = pattern[i].vertex2 not in vertex_assigments or vertex_assigments[pattern[i].vertex2] == edge.vertex2
            if valid_v1 and valid_v2:
              vertex_assigments[pattern[i].vertex1] = edge.vertex1
              vertex_assigments[pattern[i].vertex2] = edge.vertex2
            else:
              valid_ordering = False
              break
          else:
            valid_ordering = False
            break
        if valid_ordering:
          return True
      return False
    return False


In [28]:
def seq_function(accumulator:list[EdgeCombination], edge:Edge, pattern_vertex_count:int, pattern:list[Edge]=pattern):
  for i in range(len(accumulator)):
    combination = accumulator[i]
    if combination.check_edge(edge):
      new_combination = copy.deepcopy(combination)
      new_combination.add_edge(edge)
      accumulator.append(new_combination)

  new_combination = EdgeCombination(pattern_vertex_count, len(pattern))
  new_combination.add_edge(edge)
  accumulator.append(new_combination)
  return accumulator

def comb_function(accumulator1:list[EdgeCombination], accumulator2:list[EdgeCombination], pattern:list[Edge]=pattern):
  for combination in accumulator2:
    if len(combination.edges) == 1:
      edge =  combination.edges[0]
      for i in range(len(accumulator1)):
        target_combination = accumulator1[i]
        if target_combination.check_edge(edge):
          new_combination = copy.deepcopy(target_combination)
          new_combination.add_edge(edge)
          accumulator1.append(new_combination)
  accumulator1.extend(accumulator2)
  return accumulator1

def detect_matches(combinations:list[EdgeCombination], pattern:list[Edge]):
  results = []
  for combination in combinations:
    if len(combination.edges) == len(pattern):
      if combination.check_if_pattern_is_matched(pattern):
        results.append(combination.vertices)
  return results

In [29]:
def search_pattern(rdd, pattern, b=3):
  pattern_vertex_count = process_pattern(pattern)
  return rdd.flatMap(lambda x: calculate_key(Edge(*x), pattern, b)).aggregateByKey([],lambda x, y: seq_function(x, y, pattern_vertex_count), comb_function).flatMap(lambda x: detect_matches(x[1], pattern)).collect()

In [30]:
#rdd.flatMap(lambda x: calculate_key(Edge(*x), pattern_edge_list)).groupByKey().collect()
#rdd.flatMap(lambda x: calculate_key(Edge(*x), pattern)).aggregateByKey([],seq_function, comb_function).flatMap(lambda x: detect_matches(x[1], pattern)).collect()
search_pattern(rdd, pattern)

[[2, 3, 4], [2, 3, 4], [2, 3, 4], [1, 3, 4], [1, 3, 4], [1, 3, 4]]

# Ejemplo de citas de papers

In [32]:
#### papers que citan a otro paper

cites = pd.read_csv('cora/cora.cites',sep='\t',header=None,names=['target','source'])
cites

,target,source
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960
...,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289
5427,853155,853118


In [33]:
cites.insert(1, "Cited_by", "Cited_by")
cites

,target,Cited_by,source
0,35,Cited_by,1033
1,35,Cited_by,103482
2,35,Cited_by,103515
3,35,Cited_by,1050679
4,35,Cited_by,1103960
...,...,...,...
5424,853116,Cited_by,19621
5425,853116,Cited_by,853155
5426,853118,Cited_by,1140289
5427,853155,Cited_by,853118


In [34]:
df = spark.createDataFrame(cites)
df

DataFrame[target: bigint, Cited_by: string, source: bigint]

In [35]:
new_rdd = df.rdd.map(lambda x: (x["target"], x["Cited_by"], x["source"]))
new_rdd.collect()

[(35, 'Cited_by', 1033),
 (35, 'Cited_by', 103482),
 (35, 'Cited_by', 103515),
 (35, 'Cited_by', 1050679),
 (35, 'Cited_by', 1103960),
 (35, 'Cited_by', 1103985),
 (35, 'Cited_by', 1109199),
 (35, 'Cited_by', 1112911),
 (35, 'Cited_by', 1113438),
 (35, 'Cited_by', 1113831),
 (35, 'Cited_by', 1114331),
 (35, 'Cited_by', 1117476),
 (35, 'Cited_by', 1119505),
 (35, 'Cited_by', 1119708),
 (35, 'Cited_by', 1120431),
 (35, 'Cited_by', 1123756),
 (35, 'Cited_by', 1125386),
 (35, 'Cited_by', 1127430),
 (35, 'Cited_by', 1127913),
 (35, 'Cited_by', 1128204),
 (35, 'Cited_by', 1128227),
 (35, 'Cited_by', 1128314),
 (35, 'Cited_by', 1128453),
 (35, 'Cited_by', 1128945),
 (35, 'Cited_by', 1128959),
 (35, 'Cited_by', 1128985),
 (35, 'Cited_by', 1129018),
 (35, 'Cited_by', 1129027),
 (35, 'Cited_by', 1129573),
 (35, 'Cited_by', 1129683),
 (35, 'Cited_by', 1129778),
 (35, 'Cited_by', 1130847),
 (35, 'Cited_by', 1130856),
 (35, 'Cited_by', 1131116),
 (35, 'Cited_by', 1131360),
 (35, 'Cited_by', 1131557

In [36]:
paper_pattern = [Edge('a', "Cited_by", 'b'), Edge('b', "Cited_by", 'c'), Edge('c', "Cited_by", 'd')]

search_pattern(new_rdd, paper_pattern, 30)

[[126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [205196, 628766, 815096],
 [12576, 416455, 83725],
 [12576, 83725, 8875],
 [126927, 645897, 643239],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [103543, 126927, 645897],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [210871, 273152, 210872],
 [184918, 171225, 330148],
 [32083, 346292, 284023],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [159085, 1152711, 241821],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [6151, 6213, 161221],
 [6151, 6213, 195361],
 [6151, 6213, 28471],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],
 [153063, 1131223, 561613],
 [10177, 15429, 1104007],
 [2440, 18615, 4660],
 [126927, 645897, 644577],
 [644427, 644577, 645897, 126927],